# Library

In [77]:
# Native library
import math
import json
import joblib

import os
import sys
path = os.path.join(os.pardir, os.pardir) # 'crop-forecasting'
sys.path.append(path)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr

# Data prepocessing
from src.features.datascaler import DatasetScaler
from sklearn.preprocessing import MinMaxScaler

import plotly.express as px

# Model
from sklearn.cluster import KMeans
import hdbscan
from umap import UMAP

# Constant

In [20]:
# Target variable
TARGET = "Rice Yield (kg/ha)"
TARGET_TEST = 'Predicted Rice Yield (kg/ha)'
S_COLUMNS = ['ndvi', 'savi', 'evi', 'rep', 'osavi', 'rdvi', 'mtvi1', 'lswi']
G_COLUMNS = ['Field size (ha)', 'Rice Crop Intensity(D=Double, T=Triple)']
M_COLUMNS = ['tempmax', 'tempmin', 'temp', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover', 'windspeed', 'winddir', 
             'sealevelpressure', 'cloudcover', 'solarradiation', 'solarenergy', 'uvindex', 'moonphase', 'solarexposure']
FOLDER = 'augment_10_5'

INTERRIM_DIR = os.path.join(os.pardir, os.pardir, 'data', 'interim', FOLDER)
DATA_DIR = os.path.join(os.pardir, os.pardir, 'data', 'processed', FOLDER)
DATA_RAW_DIR = os.path.join(os.pardir, os.pardir, 'data', 'raw')

# Load Data

In [21]:
xdf = xr.open_dataset(os.path.join(DATA_DIR, 'train.nc'))
xdf_test = xr.open_dataset(os.path.join(DATA_DIR, 'test.nc'))
xdf

<xarray.Dataset>
Dimensions:                                  (ts_obs: 557, state_dev: 24,
                                              ts_aug: 10, datetime: 275, name: 3)
Coordinates:
    ts_id                                    (ts_obs, ts_aug) int32 ...
  * ts_obs                                   (ts_obs) int32 0 1 2 ... 555 556
  * datetime                                 (datetime) datetime64[ns] 2021-1...
  * state_dev                                (state_dev) int32 23 22 21 ... 1 0
  * ts_aug                                   (ts_aug) int32 0 1 2 3 4 5 6 7 8 9
  * name                                     (name) object 'Chau_Phu' ... 'Th...
Data variables: (12/30)
    time                                     (ts_obs, state_dev, ts_aug) datetime64[ns] ...
    ndvi                                     (ts_obs, ts_aug, state_dev) float32 ...
    savi                                     (ts_obs, ts_aug, state_dev) float32 ...
    evi                                      (ts_obs, ts_aug, state_dev) float32 ...
    rep                                      (ts_obs, ts_aug, state_dev) float32 ...
    osavi                                    (ts_obs, ts_aug, state_dev) float32 ...
    ...                                       ...
    cloudcover                               (datetime, name) float64 ...
    solarradiation                           (datetime, name) float64 ...
    solarenergy                              (datetime, name) float64 ...
    uvindex                                  (datetime, name) float64 ...
    moonphase                                (datetime, name) float64 ...
    solarexposure                            (datetime, name) float64 ...

# Format Data

## Inverse Transform

In [22]:
def inverse_transform(xdf: xr.Dataset, target: str)->xr.Dataset:
    scaler: DatasetScaler = joblib.load(os.path.join(DATA_DIR, 'scaler_dataset.joblib'))
    xdf = scaler.inverse_transform(xdf, target)
    xdf = xr.merge([xdf[G_COLUMNS], xdf[M_COLUMNS].sel(datetime=xdf['time'], name=xdf['District']), xdf[S_COLUMNS], xdf[[target]]])
    # xdf = xdf.mean(dim="ts_aug", skipna=True)
    xdf = xdf.drop(['name', 'datetime'])
    return xdf
 
xdf = inverse_transform(xdf, TARGET)
xdf_test = inverse_transform(xdf_test, TARGET_TEST)
xdf

<xarray.Dataset>
Dimensions:                                  (ts_obs: 557, state_dev: 24,
                                              ts_aug: 10)
Coordinates:
  * ts_obs                                   (ts_obs) int64 0 1 2 ... 555 556
  * ts_aug                                   (ts_aug) int64 0 1 2 3 4 5 6 7 8 9
  * state_dev                                (state_dev) int64 23 22 21 ... 1 0
    ts_id                                    (ts_obs, ts_aug) int32 0 1 ... 5569
Data variables: (12/28)
    Field size (ha)                          (ts_obs) float64 3.4 2.43 ... 2.3
    Rice Crop Intensity(D=Double, T=Triple)  (ts_obs) float64 3.0 3.0 ... 3.0
    tempmax                                  (ts_obs, state_dev, ts_aug) float64 ...
    tempmin                                  (ts_obs, state_dev, ts_aug) float64 ...
    temp                                     (ts_obs, state_dev, ts_aug) float64 ...
    dew                                      (ts_obs, state_dev, ts_aug) float64 ...
    ...                                       ...
    rep                                      (ts_obs, ts_aug, state_dev) float64 ...
    osavi                                    (ts_obs, ts_aug, state_dev) float64 ...
    rdvi                                     (ts_obs, ts_aug, state_dev) float64 ...
    mtvi1                                    (ts_obs, ts_aug, state_dev) float64 ...
    lswi                                     (ts_obs, ts_aug, state_dev) float64 ...
    Rice Yield (kg/ha)                       (ts_obs) float64 5.5e+03 ... 7.2...

## Dataset TO DataFrame

In [23]:
def ds2df(xdf: xr.Dataset, target: str)->pd.DataFrame:
    df = xdf.to_dataframe()
    df.set_index(G_COLUMNS + [target], append=True, inplace=True)
    df.reset_index('state_dev', inplace=True)
    df['state_dev'] = df['state_dev'].astype(str)
    df = df.pivot(columns='state_dev')
    df.columns = df.columns.map('_'.join).str.strip('_')
    df.reset_index(G_COLUMNS, inplace=True)
    return df

df = ds2df(xdf, TARGET)
df_test = ds2df(xdf_test, TARGET_TEST)
df

Field size (ha)  \
ts_obs ts_aug Rice Yield (kg/ha)                    
0      0      5500.0                          3.4   
       1      5500.0                          3.4   
       2      5500.0                          3.4   
       3      5500.0                          3.4   
       4      5500.0                          3.4   
...                                           ...   
556    5      7200.0                          2.3   
       6      7200.0                          2.3   
       7      7200.0                          2.3   
       8      7200.0                          2.3   
       9      7200.0                          2.3   

                                  Rice Crop Intensity(D=Double, T=Triple)  \
ts_obs ts_aug Rice Yield (kg/ha)                                            
0      0      5500.0                                                  3.0   
       1      5500.0                                                  3.0   
       2      5500.0                                                  3.0   
       3      5500.0                                                  3.0   
       4      5500.0                                                  3.0   
...                                                                   ...   
556    5      7200.0                                                  3.0   
       6      7200.0                                                  3.0   
       7      7200.0                                                  3.0   
       8      7200.0                                                  3.0   
       9      7200.0                                                  3.0   

                                  tempmax_0  tempmax_1  tempmax_10  \
ts_obs ts_aug Rice Yield (kg/ha)                                     
0      0      5500.0                   32.0       32.0        30.6   
       1      5500.0                   32.0       32.0        30.6   
       2      5500.0                   32.0       32.0        30.6   
       3      5500.0                   32.0       32.0        30.6   
       4      5500.0                   32.0       32.0        30.6   
...                                     ...        ...         ...   
556    5      7200.0                   29.3       29.0        32.0   
       6      7200.0                   29.3       29.0        32.0   
       7      7200.0                   29.3       29.0        32.0   
       8      7200.0                   29.3       29.0        32.0   
       9      7200.0                   29.3       29.0        32.0   

                                  tempmax_11  tempmax_12  tempmax_13  \
ts_obs ts_aug Rice Yield (kg/ha)                                       
0      0      5500.0                    33.0        32.0        32.0   
       1      5500.0                    33.0        32.0        32.0   
       2      5500.0                    33.0        32.0        32.0   
       3      5500.0                    33.0        32.0        32.0   
       4      5500.0                    33.0        32.0        32.0   
...                                      ...         ...         ...   
556    5      7200.0                    32.0        33.0        32.3   
       6      7200.0                    32.0        33.0        32.3   
       7      7200.0                    32.0        33.0        32.3   
       8      7200.0                    32.0        33.0        32.3   
       9      7200.0                    32.0        33.0        32.3   

                                  tempmax_14  tempmax_15  ...   lswi_21  \
ts_obs ts_aug Rice Yield (kg/ha)                          ...             
0      0      5500.0                    31.0        30.1  ...  0.163010   
       1      5500.0                    31.0        30.1  ...  0.168801   
       2      5500.0                    31.0        30.1  ...  0.162941   
       3      5500.0                    31.0        30.1  ...  0.176617   
       4      5500.0                    31.0

## Add Raw Data

In [24]:
def add_raw(df: pd.DataFrame, test: bool)->pd.DataFrame:
    if test:
        df_raw = pd.read_csv(os.path.join(DATA_RAW_DIR, 'test.csv'))
    else:
        df_raw = pd.read_csv(os.path.join(DATA_RAW_DIR, 'train.csv'))
    df = pd.merge(df, df_raw[['District', 'Latitude', 'Longitude', 'Date of Harvest']], right_index=True, left_on='ts_obs')
    return df

df = add_raw(df, False)
df_test = add_raw(df_test, True)
df

Field size (ha)  \
ts_obs ts_aug Rice Yield (kg/ha)                    
0      0      5500.0                          3.4   
       1      5500.0                          3.4   
       2      5500.0                          3.4   
       3      5500.0                          3.4   
       4      5500.0                          3.4   
...                                           ...   
556    5      7200.0                          2.3   
       6      7200.0                          2.3   
       7      7200.0                          2.3   
       8      7200.0                          2.3   
       9      7200.0                          2.3   

                                  Rice Crop Intensity(D=Double, T=Triple)  \
ts_obs ts_aug Rice Yield (kg/ha)                                            
0      0      5500.0                                                  3.0   
       1      5500.0                                                  3.0   
       2      5500.0                                                  3.0   
       3      5500.0                                                  3.0   
       4      5500.0                                                  3.0   
...                                                                   ...   
556    5      7200.0                                                  3.0   
       6      7200.0                                                  3.0   
       7      7200.0                                                  3.0   
       8      7200.0                                                  3.0   
       9      7200.0                                                  3.0   

                                  tempmax_0  tempmax_1  tempmax_10  \
ts_obs ts_aug Rice Yield (kg/ha)                                     
0      0      5500.0                   32.0       32.0        30.6   
       1      5500.0                   32.0       32.0        30.6   
       2      5500.0                   32.0       32.0        30.6   
       3      5500.0                   32.0       32.0        30.6   
       4      5500.0                   32.0       32.0        30.6   
...                                     ...        ...         ...   
556    5      7200.0                   29.3       29.0        32.0   
       6      7200.0                   29.3       29.0        32.0   
       7      7200.0                   29.3       29.0        32.0   
       8      7200.0                   29.3       29.0        32.0   
       9      7200.0                   29.3       29.0        32.0   

                                  tempmax_11  tempmax_12  tempmax_13  \
ts_obs ts_aug Rice Yield (kg/ha)                                       
0      0      5500.0                    33.0        32.0        32.0   
       1      5500.0                    33.0        32.0        32.0   
       2      5500.0                    33.0        32.0        32.0   
       3      5500.0                    33.0        32.0        32.0   
       4      5500.0                    33.0        32.0        32.0   
...                                      ...         ...         ...   
556    5      7200.0                    32.0        33.0        32.3   
       6      7200.0                    32.0        33.0        32.3   
       7      7200.0                    32.0        33.0        32.3   
       8      7200.0                    32.0        33.0        32.3   
       9      7200.0                    32.0        33.0        32.3   

                                  tempmax_14  tempmax_15  ...    lswi_4  \
ts_obs ts_aug Rice Yield (kg/ha)                          ...             
0      0      5500.0                    31.0        30.1  ...  0.166488   
       1      5500.0                    31.0        30.1  ...  0.139432   
       2      5500.0                    31.0        30.1  ...  0.153721   
       3      5500.0                    31.0        30.1  ...  0.134005   
       4      5500.0                    31.0

## Categorical Encoding

In [25]:
def encode_categorical(df: pd.DataFrame)->pd.DataFrame:
    df['Date of Harvest'] = pd.to_datetime(df['Date of Harvest']).dt.month
    df['Date of Harvest'] = MinMaxScaler((0, 2 * math.pi)).fit_transform(df['Date of Harvest'].to_numpy().reshape(-1, 1)).reshape(-1)
    df['Sin Date of Harvest'] = np.sin(df['Date of Harvest'])
    df['Cos Date of Harvest'] = np.cos(df['Date of Harvest'])
    df.drop(columns=['Date of Harvest'], inplace=True)
    df = pd.get_dummies(df, columns=['District'], drop_first=True)
    return df

df = encode_categorical(df)
df_test = encode_categorical(df_test)
df

Field size (ha)  \
ts_obs ts_aug Rice Yield (kg/ha)                    
0      0      5500.0                          3.4   
       1      5500.0                          3.4   
       2      5500.0                          3.4   
       3      5500.0                          3.4   
       4      5500.0                          3.4   
...                                           ...   
556    5      7200.0                          2.3   
       6      7200.0                          2.3   
       7      7200.0                          2.3   
       8      7200.0                          2.3   
       9      7200.0                          2.3   

                                  Rice Crop Intensity(D=Double, T=Triple)  \
ts_obs ts_aug Rice Yield (kg/ha)                                            
0      0      5500.0                                                  3.0   
       1      5500.0                                                  3.0   
       2      5500.0                                                  3.0   
       3      5500.0                                                  3.0   
       4      5500.0                                                  3.0   
...                                                                   ...   
556    5      7200.0                                                  3.0   
       6      7200.0                                                  3.0   
       7      7200.0                                                  3.0   
       8      7200.0                                                  3.0   
       9      7200.0                                                  3.0   

                                  tempmax_0  tempmax_1  tempmax_10  \
ts_obs ts_aug Rice Yield (kg/ha)                                     
0      0      5500.0                   32.0       32.0        30.6   
       1      5500.0                   32.0       32.0        30.6   
       2      5500.0                   32.0       32.0        30.6   
       3      5500.0                   32.0       32.0        30.6   
       4      5500.0                   32.0       32.0        30.6   
...                                     ...        ...         ...   
556    5      7200.0                   29.3       29.0        32.0   
       6      7200.0                   29.3       29.0        32.0   
       7      7200.0                   29.3       29.0        32.0   
       8      7200.0                   29.3       29.0        32.0   
       9      7200.0                   29.3       29.0        32.0   

                                  tempmax_11  tempmax_12  tempmax_13  \
ts_obs ts_aug Rice Yield (kg/ha)                                       
0      0      5500.0                    33.0        32.0        32.0   
       1      5500.0                    33.0        32.0        32.0   
       2      5500.0                    33.0        32.0        32.0   
       3      5500.0                    33.0        32.0        32.0   
       4      5500.0                    33.0        32.0        32.0   
...                                      ...         ...         ...   
556    5      7200.0                    32.0        33.0        32.3   
       6      7200.0                    32.0        33.0        32.3   
       7      7200.0                    32.0        33.0        32.3   
       8      7200.0                    32.0        33.0        32.3   
       9      7200.0                    32.0        33.0        32.3   

                                  tempmax_14  tempmax_15  ...    lswi_6  \
ts_obs ts_aug Rice Yield (kg/ha)                          ...             
0      0      5500.0                    31.0        30.1  ...  0.104435   
       1      5500.0                    31.0        30.1  ...  0.074214   
       2      5500.0                    31.0        30.1  ...  0.078648   
       3      5500.0                    31.0        30.1  ...  0.094775   
       4      5500.0                    31.0

## Concat Data

In [26]:
df = pd.concat([df, df_test], axis='index')

# Clustering

## KMeans

In [73]:
kmeans = KMeans(n_clusters=6)
pred = kmeans.fit_predict(df)

In [74]:
df_cluster = pd.DataFrame(pred.T, index=df.index, columns=['Cluster'])
df_cluster.reset_index(inplace=True)
test_bool = df_cluster[TARGET].isna()
df_cluster['Dataset'] = 'Train'
df_cluster['Dataset'][test_bool] = 'Test'
df_cluster['Cluster'] = df_cluster['Cluster'].astype(str)
fig = px.histogram(df_cluster, x='Cluster', color='Dataset', histnorm='probability', text_auto=True)
fig.update_layout(
    title='Repartition into cluster of Train/Test Dataset in percentage'
)

In [76]:
df_cluster_train = df_cluster[(df_cluster['Dataset'] == 'Train') & (df_cluster['Cluster'] == '5')].copy(deep=True)
df_cluster_train['ts_obs'] = df_cluster_train['ts_obs'].astype(str)

fig = px.histogram(df_cluster_train, x='ts_obs')
fig.update_layout(
    title='Repartition into Observation ID of the Cluster number 1'
)
fig.show()

df_cluster_train['ts_obs'] = df_cluster_train['ts_obs'].astype(int)

## UMAP

In [68]:
reducer = UMAP(n_components=3)
embedding = reducer.fit_transform(df)
embedding.shape

(6570, 3)

In [69]:
df_cluster = pd.DataFrame(embedding, index=df.index, columns=['x', 'y', 'z'])
df_cluster.reset_index(inplace=True)
test_bool = df_cluster[TARGET].isna()
df_cluster['Dataset'] = 'Train'
df_cluster['Dataset'][test_bool] = 'Test'

fig = px.scatter_3d(df_cluster, x='x', y='y', z='z', color='Dataset')
fig.update_traces(marker_size = 4)
fig.update_layout(title='Data representation after UMAP application on 3 Dimensions')
fig.show()

In [70]:
labels = hdbscan.HDBSCAN(
    min_samples=10,
    min_cluster_size=500,
).fit_predict(embedding)

Cluster
 4         1440
 2         1289
 1          951
 3          870
 5          840
 0          700
-1          480
dtype: int64

In [88]:
cluster_pred = pd.DataFrame(labels.T, index=df.index, columns=['Cluster'])
df_cluster.reset_index(inplace=True)
test_bool = df_cluster[TARGET].isna()
df_cluster['Dataset'] = 'Train'
df_cluster['Dataset'][test_bool] = 'Test'
df_cluster['Cluster'] = df_cluster['Cluster'].astype(str)
fig = px.histogram(df_cluster, x='Cluster', color='Dataset', histnorm='probability', text_auto=True)
fig.update_layout(
    title='Repartition into cluster of Train/Test Dataset in percentage'
)

In [94]:
df_cluster_train = df_cluster[(df_cluster['Dataset'] == 'Train') & (df_cluster['Cluster'] == '5')].copy(deep=True)
df_cluster_train['ts_obs'] = df_cluster_train['ts_obs'].astype(str)

fig = px.histogram(df_cluster_train, x='ts_obs')
fig.update_layout(
    title='Repartition into Observation ID of the Cluster number 1'
)
fig.show()

df_cluster_train['ts_obs'] = df_cluster_train['ts_obs'].astype(int)

In [95]:
ts_obs_0 = df_cluster_train['ts_obs'].unique()
len(ts_obs_0)

95

In [96]:
df_cluster_train = df_cluster[(df_cluster['Dataset'] == 'Train') & (df_cluster['Cluster'] == '3')].copy(deep=True)
df_cluster_train['ts_obs'] = df_cluster_train['ts_obs'].astype(str)

fig = px.histogram(df_cluster_train, x='ts_obs')
fig.update_layout(
    title='Repartition into Observation ID of the Cluster number 1'
)
fig.show()

df_cluster_train['ts_obs'] = df_cluster_train['ts_obs'].astype(int)

In [97]:
ts_obs_1 = df_cluster_train['ts_obs'].unique()
len(ts_obs_1)

73

In [102]:
ts_obs = list(df_cluster_train['ts_obs'])
index = {
    'dimensionality_reduction': 'UMAP',
    'clustering': 'HDBSCAN',
    'clusters': [
        {
            'train': {'ts_obs': df_cluster[(df_cluster['Dataset'] == 'Train') & (df_cluster['Cluster'] == '5')]['ts_obs'].unique().tolist()},
            'test': {'ts_obs': df_cluster[(df_cluster['Dataset'] == 'Test') & (df_cluster['Cluster'] == '5')]['ts_obs'].unique().tolist()}
        },
        {
            'train': {'ts_obs': df_cluster[(df_cluster['Dataset'] == 'Train') & (df_cluster['Cluster'] == '3')]['ts_obs'].unique().tolist()},
            'test': {'ts_obs': df_cluster[(df_cluster['Dataset'] == 'Test') & (df_cluster['Cluster'] == '3')]['ts_obs'].unique().tolist()}
        }
    ]
}

with open(os.path.join(DATA_DIR, 'index.json'), 'w') as f:
    json.dump(index, f)
with open(os.path.join(INTERRIM_DIR, 'index.json'), 'w') as f:
    json.dump(index, f)